In [ ]:
using JuMP
using Clp

m = Model(solver = ClpSolver())
@variable(m, 0 <= x <= 2 )
@variable(m, 0 <= y <= 30 )

@objective(m, Max, 5x + 3*y )
@constraint(m, 1x + 5y <= 3.0 )

print(m)

status = solve(m)

println("Objective value: ", getobjectivevalue(m))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

In [ ]:
n=3
W=zeros(Int,n,n)
for row in 1:n,col in 1:n
    W[row,col] = (row-1)*n+col
end
println(W)



In [ ]:
println(abs(-1))

In [6]:
using JuMP
# using Gurobi
using GLPK
using GLPKMathProgInterface


# Given a n-by-n matrix representing the solution to an undirected TSP,
# extract the tour as a vector
# Input:
#  n        Number of cities
#  sol      n-by-n 0-1 symmetric matrix representing solution
# Output:
#  tour     n+1 length vector of tour, starting and ending at 1
function extractTour(n, sol,origin)
    tour = [origin]  # Start at city 1 always
    cur_city = origin
    while true
        # Look for first arc out of current city
        for j = 1:n
            if sol[cur_city,j] >= 1-1e-6
                # Found next city
                push!(tour, j)
                # Don't ever use this arc again
                sol[cur_city, j] = 0.0
                sol[j, cur_city] = 0.0
                # Move to next city
                cur_city = j
                break
            end
        end
        # If we have come back to 1, stop
        if cur_city == origin
            break
        end
    end  # end while
    return tour
end

# solveTSP
# Given a matrix of city locations, solve the TSP
# Inputs:
#   n       Number of cities
#   cities  n-by-2 matrix of (x,y) city locations
# Output:
#   path    Vector with order to cities are visited in
function solveTSP(n, cities)

    # Calculate pairwise distance matrix
    dist = zeros(n, n)
    for i = 1:n
        for j = i:n
            d = norm(cities[i,1:2] - cities[j,1:2])
            dist[i,j] = d
            dist[j,i] = d
        end
    end

    # Create a model that will use Gurobi to solve
    m = Model(solver=GLPKSolverMIP())

    # x[i,j] is 1 iff we travel between i and j, 0 otherwise
    # Although we define all n^2 variables, we will only use
    # the upper triangle
    @variable(m, x[1:n,1:n], Bin)

    # Minimize length of tour
    @objective(m, Min, sum(dist[i,j]*x[i,j] for i=1:n for j=i:n))

    # Make x_ij and x_ji be the same thing (undirectional)
    # Don't allow self-arcs
    for i = 1:n
        @constraint(m, x[i,i] == 0)
        for j = (i+1):n
            @constraint(m, x[i,j] == x[j,i])
        end
    end

    # We must enter and leave every city once and only once
    for i = 1:n
        @constraint(m, sum(x[i,j] for j=1:n) == 2)
    end

    
    solve(m)

    # Return best tour
    # return extractTour(n, getvalue(x))
    return x
end  # end solveTSP


# Create a simple instance that looks like
#       +           +
#   +                   +
#       +           +
# The optimal tour is obvious, but the initial solution will be
#    /--+           +--\
#   +               |   +
#    \--+           +--/
n = 6
cities = [ 50 2000000000;
                    100 100;
                    100 300;
                    500 100;
                    500 300;
                    550 200000000]
tour = solveTSP(n, cities)
println("Solution: ")
println(extractTour(n, getvalue(tour),3))

Solution: 
[3,1,6,5,4,2,3]


In [7]:
println(extractTour(n, getvalue(tour),1))
println(extractTour(n, getvalue(tour),3))
println(extractTour(n, getvalue(tour),6))


[1,3,2,4,5,6,1]
[3,1,6,5,4,2,3]
[6,1,3,2,4,5,6]
